In [ ]:
from scipy import misc, ndimage, signal
from sklearn.model_selection  import train_test_split
import numpy
import numpy as np
import random
import ntpath
import os
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from keras import optimizers 
from keras import regularizers
import tensorflow as tf
import cv2
from time import time
import time as tm
import datetime
from operator import itemgetter
import glob
from skimage.util.shape import view_as_blocks
from keras.utils import np_utils
from keras.utils import to_categorical 

#USE OF CPU, WHEN IS NOT AVAILABLE THE GPU
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
if tf.test.gpu_device_name():
    print('GPU found')
else:
    print("No GPU found")

In [ ]:
!nvidia-smi
#!kill PIDnumber

In [ ]:
path_folder = "./"
path_img_base = './images/YE'
if not os.path.exists(path_img_base):
    os.makedirs(path_img_base)

In [ ]:
################################################## 30 original SRM FILTERS
srm_weights = np.load(path_folder+'/SRM_Kernels1.npy') 
print (srm_weights.shape)
################################################## bias
biasSRM=numpy.ones(30)
################################################## TLU ACTIVATION FUNCTION
def Tanh3(x):
    tanh3 = tf.keras.activations.tanh(x)*3
    return tanh3
##################################################

In [ ]:
def get_ye(img_size=256):
    tf.keras.backend.clear_session()
    
    #Inputs
    inputs = tf.keras.Input(shape=(img_size,img_size,1), name="input_1")
    print(inputs.shape)
    
    #Block 1
    layers = tf.keras.layers.Conv2D(30, (5,5), weights=[srm_weights,biasSRM], strides=(1,1), trainable=False, activation=Tanh3, use_bias=True)(inputs)
    print(layers.shape)
    
    #Block 2
    
    layers = tf.keras.layers.Conv2D(30, (3,3), strides=(1,1), kernel_initializer='glorot_normal',kernel_regularizer=tf.keras.regularizers.l2(0.0001),bias_regularizer=tf.keras.regularizers.l2(0.0001))(layers) 
    layers = tf.keras.layers.ReLU(negative_slope=0.1, threshold=0)(layers)
    layers = tf.keras.layers.Lambda(tf.keras.backend.abs)(layers)
    layers = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layers)
    layers = tf.keras.layers.Concatenate()([layers, layers, layers])
    print(layers.shape)
    
    #Block 3
    layers = tf.keras.layers.SpatialDropout2D(rate=0.1)(layers)
    layers = tf.keras.layers.Conv2D(30, (3,3), strides=(1,1), kernel_initializer='glorot_normal',kernel_regularizer=tf.keras.regularizers.l2(0.0001),bias_regularizer=tf.keras.regularizers.l2(0.0001))(layers) 
    layers = tf.keras.layers.ReLU(negative_slope=0.1, threshold=0)(layers)
    layers = tf.keras.layers.Lambda(tf.keras.backend.abs)(layers)
    layers = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layers)
    print(layers.shape)
    
    #Block 4
    layers = tf.keras.layers.SpatialDropout2D(rate=0.1)(layers)
    layers = tf.keras.layers.Conv2D(30, (3,3), strides=(1,1), kernel_initializer='glorot_normal',kernel_regularizer=tf.keras.regularizers.l2(0.0001),bias_regularizer=tf.keras.regularizers.l2(0.0001))(layers) 
    layers = tf.keras.layers.ReLU(negative_slope=0.1, threshold=0)(layers)
    layers = tf.keras.layers.Lambda(tf.keras.backend.abs)(layers)
    layers = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layers)
    layers = tf.keras.layers.AveragePooling2D((2,2), strides= (2,2))(layers)
    print(layers.shape)
    
    #Block 5
    layers = tf.keras.layers.SpatialDropout2D(rate=0.1)(layers)
    layers = tf.keras.layers.Conv2D(32, (5,5), strides=(1,1), kernel_initializer='glorot_normal',kernel_regularizer=tf.keras.regularizers.l2(0.0001),bias_regularizer=tf.keras.regularizers.l2(0.0001))(layers)
    layers = tf.keras.layers.ReLU(negative_slope=0.1, threshold=0)(layers)
    layers = tf.keras.layers.Lambda(tf.keras.backend.abs)(layers)
    layers = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layers)
    print(layers.shape)
    
    #Block 6
    layers = tf.keras.layers.Concatenate()([layers, layers, layers])
    layers = tf.keras.layers.GlobalAveragePooling2D(data_format="channels_last")(layers)
    layers = tf.keras.layers.Dense(128,kernel_initializer='glorot_normal',kernel_regularizer=tf.keras.regularizers.l2(0.0001),bias_regularizer=tf.keras.regularizers.l2(0.0001))(layers)
    layers = tf.keras.layers.ReLU(negative_slope=0.1, threshold=0)(layers)
    layers = tf.keras.layers.Dense(64,kernel_initializer='glorot_normal',kernel_regularizer=tf.keras.regularizers.l2(0.0001),bias_regularizer=tf.keras.regularizers.l2(0.0001))(layers)
    layers = tf.keras.layers.ReLU(negative_slope=0.1, threshold=0)(layers)
    layers = tf.keras.layers.Dense(32,kernel_initializer='glorot_normal',kernel_regularizer=tf.keras.regularizers.l2(0.0001),bias_regularizer=tf.keras.regularizers.l2(0.0001))(layers)
    layers = tf.keras.layers.ReLU(negative_slope=0.1, threshold=0)(layers)
    predictions = tf.keras.layers.Dense(2,kernel_initializer='glorot_normal', activation="softmax", name="output_1",kernel_regularizer=tf.keras.regularizers.l2(0.0001),bias_regularizer=tf.keras.regularizers.l2(0.0001))(layers)
    print(predictions.shape)
    
    #Model generation
    model = tf.keras.Model(inputs = inputs, outputs=predictions)
    #Optimizer
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005, momentum=0.95)#lrate
    #Compilator
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    print ("Ye-net model 2 generated")
    return model


In [ ]:
model_untrained = get_ye(img_size=256)

In [ ]:
path_model_trained2 = path_folder+"/YE_WOW04.hdf5"
model_trained2 = tf.keras.models.load_model(path_model_trained2, custom_objects={'Tanh3':Tanh3}, compile=True)

In [ ]:
path_stego = "/media/ia/Datos1/DocReinel/Steganalysis/H_Brayan_A_Arteaga/Frankenstein_Project/GBRAS-Net/DATABASES/BOSSbase-1.01/WOW/0.4bpp/stego"
path_cover = "/media/ia/Datos1/DocReinel/Steganalysis/H_Brayan_A_Arteaga/Frankenstein_Project/GBRAS-Net/DATABASES/BOSSbase-1.01/cover"

n=256
def load_images(path_pattern):
    files=glob.glob(path_pattern)
    X=[]
    for f in sorted(files):
        I = cv2.imread(f, cv2.IMREAD_GRAYSCALE)
        patches = view_as_blocks(I, (n, n))
        for i in range(patches.shape[0]):
            for j in range(patches.shape[1]):
                X.append( [ patches[i,j] ] )
    X=numpy.array(X)
    return X

Xc_ = load_images(path_cover+'/*.pgm') ##COVER IMAGES
Xs_ = load_images(path_stego+'/*.pgm') ##STEGO IMAGES
X_  = (numpy.vstack((Xc_, Xs_)))
Xt_ = (numpy.hstack(([0]*len(Xc_), [1]*len(Xs_))))
Xt_ = np_utils.to_categorical(Xt_, 2)
X_  = np.rollaxis(X_,1,4)  #channel axis shifted to last axis

print("Total image data and labels",X_.shape,Xt_.shape)
#Cover hasta las 10000 ##Train hasta las 4000 ##Valid hasta de las 4000 a las 5000 ##Test de las 5000 a las 10000
X_train = np.concatenate([X_[0:4000],X_[10000:14000]],axis=0)
X_valid = np.concatenate([X_[4000:5000],X_[14000:15000]],axis=0)
X_test  = np.concatenate([X_[5000:10000],X_[15000:20000]],axis=0)
y_train = np.concatenate([Xt_[0:4000],Xt_[10000:14000]],axis=0)
y_valid = np.concatenate([Xt_[4000:5000],Xt_[14000:15000]],axis=0)
y_test  = np.concatenate([Xt_[5000:10000],Xt_[15000:20000]],axis=0)
#Controled randomized data for training
X_dat0, X_dat1, y_dat0, y_dat1 = train_test_split(X_train, y_train, test_size=0.50, random_state=64) 
X_train = np.concatenate([X_dat0,X_dat1],axis=0) 
y_train = np.concatenate([y_dat0,y_dat1],axis=0) 
print(X_train.shape)
print(y_train.shape)
print(X_valid.shape)
print(y_valid.shape)
print(X_test.shape)
print(y_test.shape)


In [ ]:
loss,accuracy = model_untrained.evaluate(X_test,y_test,verbose=0) #Test BOSSbase 1.01 
print(f'Loss={loss:.4f} and Accuracy={accuracy:0.3f}')

loss,accuracy = model_trained2.evaluate(X_test,y_test,verbose=0) #Test BOSSbase 1.01 
print(f'Loss={loss:.4f} and Accuracy={accuracy:0.3f}')

In [ ]:
images = [943,2507]
for image in images:
    print(image)
    plt.imshow(X_test[image,:,:,0]-X_test[5000+image,:,:,0],cmap="gray")
    plt.show()

In [ ]:
image = 943
path_img_base = './images/YE/c_s_cs'
if not os.path.exists(path_img_base):
    os.makedirs(path_img_base)

plt.figure(figsize=(30,30))
plt.imshow(X_test[image,:,:,0],cmap="gray")
plt.axis("off")
plt.savefig(path_img_base+'/cover'+'.pdf', format='pdf') 
plt.savefig(path_img_base+'/cover'+'.svg', format='svg') 
plt.show()

plt.figure(figsize=(30,30))
plt.imshow(X_test[5000+image,:,:,0],cmap="gray")
plt.axis("off")
plt.savefig(path_img_base+'/stego'+'.pdf', format='pdf') 
plt.savefig(path_img_base+'/stego'+'.svg', format='svg') 
plt.show()

plt.figure(figsize=(30,30))
plt.imshow(X_test[image,:,:,0]-X_test[5000+image,:,:,0],cmap="gray")
plt.axis("off")
plt.savefig(path_img_base+'/cover_stego'+'.pdf', format='pdf') 
plt.savefig(path_img_base+'/cover_stego'+'.svg', format='svg') 
plt.show()

# Filtros

In [ ]:
path_img_base = './images/YE/SRM'
if not os.path.exists(path_img_base):
    os.makedirs(path_img_base)
    

In [ ]:
srm_weights = np.load('SRM_Kernels1.npy')
def display_filters(w, figsize=(100,100)):
    plt.figure(figsize=figsize)
    w = (w-np.min(w))/(np.max(w)-np.min(w))
    cont=0
    for i in range(w.shape[-1]):
        for j in range(w.shape[-2]):
            cont=cont+1
            plt.subplot(5,6,cont)
            plt.imshow(w[:,:,0,i], interpolation="none", cmap='gray')
            plt.axis("off")
            plt.savefig(path_img_base+'/SRMoriginals'+'.pdf', format='pdf') 
            plt.savefig(path_img_base+'/SRMoriginals'+'.svg', format='svg') 
            plt.savefig(path_img_base+'/SRMoriginals'+'.jpg', format='jpeg')   
            plt.savefig(path_img_base+'/SRMoriginals'+'.png', format='png') 

display_filters(srm_weights)
print(srm_weights.shape)  

# Mapas de activación

In [ ]:
#cover
image = X_test[943,:,:,0]
image = np.reshape(image,(1,256,256,1))                 
print(np.round(model_untrained.predict(image)))

#stego
image = X_test[5943,:,:,0]
image = np.reshape(image,(1,256,256,1))
print(np.round(model_untrained.predict(image)))   

In [ ]:
#cover
image = X_test[943,:,:,0]
image = np.reshape(image,(1,256,256,1))                 
print(np.round(model_trained2.predict(image)))

#stego
image = X_test[5943,:,:,0]
image = np.reshape(image,(1,256,256,1))
print(np.round(model_trained2.predict(image)))   

In [ ]:
path_img_base = './images/YE/NOISE'
if not os.path.exists(path_img_base):
    os.makedirs(path_img_base)

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(200,200))
fig.figsize=(2500,2500)
ax1.imshow(X_test[943,:,:,0],cmap='gray')
ax1.set_title('Cover Image',fontsize=160,y=1.02)
ax1.axis("off")
ax2.imshow(X_test[5943,:,:,0],cmap='gray')
ax2.set_title('Stego Image',fontsize=160,y=1.02)
ax2.axis("off")
ax3.imshow(X_test[943,:,:,0]-X_test[5943,:,:,0],cmap='gray')
ax3.set_title('Steganographic Noise',fontsize=160,y=1.02)
ax3.axis("off")
fig.suptitle('Cover Image - Stego Image = Steganographic Noise',y=0.65,x=0.51,fontweight="bold",fontsize=180)
plt.show()

fig.savefig(path_img_base+'/noise'+'.pdf', format='pdf')  
fig.savefig(path_img_base+'/noise'+'.svg', format='svg') 
fig.savefig(path_img_base+'/noise'+'.jpg', format='jpeg')   
fig.savefig(path_img_base+'/noise'+'.png', format='png') 

In [ ]:
for i, layer in enumerate(model_trained2.layers): 
    print(i, layer.name) 

In [ ]:
# summarize filter shapes
for layer in model_trained2.layers:
    # check for convolutional layer
    if 'conv' not in layer.name:
        continue
    if 'separable_conv2d' in layer.name: 
        continue
    # get filter weights
    filters, biases = layer.get_weights()
    print(layer.name, filters.shape) 

In [ ]:
model_trained2.summary()

In [ ]:
def visualize_conv_layer(num,layer_name,image,row,col):
    plt.figure()
    #plt.imshow(image[:,:,0],cmap='gray')
    layer_output=model.get_layer(layer_name).output
    intermediate_model=tf.keras.models.Model(inputs=model.input,outputs=layer_output)
    image=numpy.reshape(image,(1,256,256,1))
    intermediate_prediction=intermediate_model.predict(image)
    print('Shape activations maps:',intermediate_prediction.shape)
    row_size=row
    col_size=col
    img_index=0
    fig,ax=plt.subplots(row_size,col_size,figsize=(30,30))
    for row in range(0,row_size):
        for col in range(0,col_size):
            ax[row][col].imshow(intermediate_prediction[0, :, :, img_index], cmap='gray')
            img_index=img_index+1
    
    if not os.path.exists(path_images+'/'+str(num)):
        os.makedirs(path_images+'/'+str(num))
    
    plt.savefig(path_images+'/'+str(num)+'/'+'ACT-MAP_All_'+layer_name+'.eps', format='eps')
    plt.savefig(path_images+'/'+str(num)+'/'+'ACT-MAP_All_'+layer_name+'.svg', format='svg')
    plt.savefig(path_images+'/'+str(num)+'/'+'ACT-MAP_All_'+layer_name+'.pdf', format='pdf')  
    plt.show()
    
    plt.figure(figsize=(30,30))
    plt.imshow(intermediate_prediction[0, :, :, 0], cmap='gray')
    print(layer_name)
    print(intermediate_prediction[0, :, :, 0])
    plt.savefig(path_images+'/'+str(num)+'/'+'ACT-MAP_First_'+layer_name+'.eps', format='eps')
    plt.savefig(path_images+'/'+str(num)+'/'+'ACT-MAP_First_'+layer_name+'.svg', format='svg')
    plt.savefig(path_images+'/'+str(num)+'/'+'ACT-MAP_First_'+layer_name+'.pdf', format='pdf')  
    plt.show() 

In [ ]:
def visualize_conv_layer_1x2(num,layer_name,image,row=1,col=2):

    plt.figure()
    layer_output=model.get_layer(layer_name).output
    intermediate_model=tf.keras.models.Model(inputs=model.input,outputs=layer_output)
    image=numpy.reshape(image,(1,256,256,1))
    intermediate_prediction=intermediate_model.predict(image)
    print('Shape activations maps:',intermediate_prediction.shape)

    fig, (ax1, ax2) = plt.subplots(row, col, figsize=(30,30))
    fig.figsize=(2500,2500)
    ax1.imshow(intermediate_prediction[0, :, :, 0], cmap='gray')
    ax1.axis("off")
    ax2.imshow(intermediate_prediction[0, :, :, 1], cmap='gray')
    ax2.axis("off")

    if not os.path.exists(path_images+'/'+str(num)):
        os.makedirs(path_images+'/'+str(num))

    plt.savefig(path_images+'/'+str(num)+'/'+'ACT-MAP_All_'+layer_name+'.eps', format='eps')
    plt.savefig(path_images+'/'+str(num)+'/'+'ACT-MAP_All_'+layer_name+'.svg', format='svg')
    plt.savefig(path_images+'/'+str(num)+'/'+'ACT-MAP_All_'+layer_name+'.pdf', format='pdf')  
    plt.show()

    plt.figure(figsize=(30,30))
    plt.imshow(intermediate_prediction[0, :, :, 0], cmap='gray')
    print(layer_name)
    print(intermediate_prediction[0, :, :, 0])
    plt.savefig(path_images+'/'+str(num)+'/'+'ACT-MAP_First_'+layer_name+'.eps', format='eps')
    plt.savefig(path_images+'/'+str(num)+'/'+'ACT-MAP_First_'+layer_name+'.svg', format='svg')
    plt.savefig(path_images+'/'+str(num)+'/'+'ACT-MAP_First_'+layer_name+'.pdf', format='pdf')  
    plt.show() 

In [ ]:
image_cover = X_test[943] 
image_stego = X_test[5943] 

In [ ]:
image_cover_stego = image_cover-image_stego

In [ ]:
### COVER IMAGE
path_images = './images/YE/cover_stego/ACT_MAPS/NO_ENTRENADO/cover'
if not os.path.exists(path_images):
    os.makedirs(path_images)
    
model = model_untrained

image_ = image_cover
visualize_conv_layer(0,'conv2d',image_,5,6)
visualize_conv_layer(1,'conv2d_1',image_,5,6)
visualize_conv_layer(2,'re_lu',image_,5,6)
visualize_conv_layer(3,'lambda',image_,5,6)
visualize_conv_layer(4,'batch_normalization',image_,5,6)
visualize_conv_layer(5,'concatenate',image_,9,10)
visualize_conv_layer(6,'spatial_dropout2d',image_,9,10)
visualize_conv_layer(7,'conv2d_2',image_,5,6)
visualize_conv_layer(8,'re_lu_1',image_,5,6)
visualize_conv_layer(9,'lambda_1',image_,5,6)
visualize_conv_layer(10,'batch_normalization_1',image_,5,6)
visualize_conv_layer(11,'spatial_dropout2d_1',image_,5,6)
visualize_conv_layer(12,'conv2d_3',image_,5,6)
visualize_conv_layer(13,'re_lu_2',image_,5,6)
visualize_conv_layer(14,'lambda_2',image_,5,6)
visualize_conv_layer(15,'batch_normalization_2',image_,5,6)
visualize_conv_layer(16,'average_pooling2d',image_,5,6)
visualize_conv_layer(17,'spatial_dropout2d_2',image_,5,6)
visualize_conv_layer(18,'conv2d_4',image_,4,8)
visualize_conv_layer(19,'re_lu_3',image_,4,8)
visualize_conv_layer(20,'lambda_3',image_,4,8)
visualize_conv_layer(21,'batch_normalization_3',image_,4,8)
visualize_conv_layer(22,'concatenate_1',image_,8,12)


In [ ]:
### STEGO IMAGE
path_images = './images/YE/cover_stego/ACT_MAPS/NO_ENTRENADO/stego'
if not os.path.exists(path_images):
    os.makedirs(path_images)
    
model = model_untrained

image_ = image_stego
visualize_conv_layer(0,'conv2d',image_,5,6)
visualize_conv_layer(1,'conv2d_1',image_,5,6)
visualize_conv_layer(2,'re_lu',image_,5,6)
visualize_conv_layer(3,'lambda',image_,5,6)
visualize_conv_layer(4,'batch_normalization',image_,5,6)
visualize_conv_layer(5,'concatenate',image_,9,10)
visualize_conv_layer(6,'spatial_dropout2d',image_,9,10)
visualize_conv_layer(7,'conv2d_2',image_,5,6)
visualize_conv_layer(8,'re_lu_1',image_,5,6)
visualize_conv_layer(9,'lambda_1',image_,5,6)
visualize_conv_layer(10,'batch_normalization_1',image_,5,6)
visualize_conv_layer(11,'spatial_dropout2d_1',image_,5,6)
visualize_conv_layer(12,'conv2d_3',image_,5,6)
visualize_conv_layer(13,'re_lu_2',image_,5,6)
visualize_conv_layer(14,'lambda_2',image_,5,6)
visualize_conv_layer(15,'batch_normalization_2',image_,5,6)
visualize_conv_layer(16,'average_pooling2d',image_,5,6)
visualize_conv_layer(17,'spatial_dropout2d_2',image_,5,6)
visualize_conv_layer(18,'conv2d_4',image_,4,8)
visualize_conv_layer(19,'re_lu_3',image_,4,8)
visualize_conv_layer(20,'lambda_3',image_,4,8)
visualize_conv_layer(21,'batch_normalization_3',image_,4,8)
visualize_conv_layer(22,'concatenate_1',image_,8,12)


In [ ]:
### COVER IMAGE
path_images = './images/YE/cover_stego/ACT_MAPS/SI_ENTRENADO/cover'
if not os.path.exists(path_images):
    os.makedirs(path_images)
    
model = model_trained2

image_ = image_cover
visualize_conv_layer(0,'conv2d',image_,5,6)
visualize_conv_layer(1,'conv2d_1',image_,5,6)
visualize_conv_layer(2,'re_lu',image_,5,6)
visualize_conv_layer(3,'lambda',image_,5,6)
visualize_conv_layer(4,'batch_normalization',image_,5,6)
visualize_conv_layer(5,'concatenate',image_,9,10)
visualize_conv_layer(6,'spatial_dropout2d',image_,9,10)
visualize_conv_layer(7,'conv2d_2',image_,5,6)
visualize_conv_layer(8,'re_lu_1',image_,5,6)
visualize_conv_layer(9,'lambda_1',image_,5,6)
visualize_conv_layer(10,'batch_normalization_1',image_,5,6)
visualize_conv_layer(11,'spatial_dropout2d_1',image_,5,6)
visualize_conv_layer(12,'conv2d_3',image_,5,6)
visualize_conv_layer(13,'re_lu_2',image_,5,6)
visualize_conv_layer(14,'lambda_2',image_,5,6)
visualize_conv_layer(15,'batch_normalization_2',image_,5,6)
visualize_conv_layer(16,'average_pooling2d',image_,5,6)
visualize_conv_layer(17,'spatial_dropout2d_2',image_,5,6)
visualize_conv_layer(18,'conv2d_4',image_,4,8)
visualize_conv_layer(19,'re_lu_3',image_,4,8)
visualize_conv_layer(20,'lambda_3',image_,4,8)
visualize_conv_layer(21,'batch_normalization_3',image_,4,8)
visualize_conv_layer(22,'concatenate_1',image_,8,12)


In [ ]:
### STEGO IMAGE
path_images = './images/YE/cover_stego/ACT_MAPS/SI_ENTRENADO/stego'
if not os.path.exists(path_images):
    os.makedirs(path_images)
    
model = model_trained2

image_ = image_stego
visualize_conv_layer(0,'conv2d',image_,5,6)
visualize_conv_layer(1,'conv2d_1',image_,5,6)
visualize_conv_layer(2,'re_lu',image_,5,6)
visualize_conv_layer(3,'lambda',image_,5,6)
visualize_conv_layer(4,'batch_normalization',image_,5,6)
visualize_conv_layer(5,'concatenate',image_,9,10)
visualize_conv_layer(6,'spatial_dropout2d',image_,9,10)
visualize_conv_layer(7,'conv2d_2',image_,5,6)
visualize_conv_layer(8,'re_lu_1',image_,5,6)
visualize_conv_layer(9,'lambda_1',image_,5,6)
visualize_conv_layer(10,'batch_normalization_1',image_,5,6)
visualize_conv_layer(11,'spatial_dropout2d_1',image_,5,6)
visualize_conv_layer(12,'conv2d_3',image_,5,6)
visualize_conv_layer(13,'re_lu_2',image_,5,6)
visualize_conv_layer(14,'lambda_2',image_,5,6)
visualize_conv_layer(15,'batch_normalization_2',image_,5,6)
visualize_conv_layer(16,'average_pooling2d',image_,5,6)
visualize_conv_layer(17,'spatial_dropout2d_2',image_,5,6)
visualize_conv_layer(18,'conv2d_4',image_,4,8)
visualize_conv_layer(19,'re_lu_3',image_,4,8)
visualize_conv_layer(20,'lambda_3',image_,4,8)
visualize_conv_layer(21,'batch_normalization_3',image_,4,8)
visualize_conv_layer(22,'concatenate_1',image_,8,12)


In [ ]:
### COVER- STEGO IMAGE
path_images = './images/YE/cover_stego/ACT_MAPS/SI_ENTRENADO/image_cover_stego'
if not os.path.exists(path_images):
    os.makedirs(path_images)
    
model = model_trained2

image_ = image_cover_stego
visualize_conv_layer(0,'conv2d',image_,5,6)
visualize_conv_layer(1,'conv2d_1',image_,5,6)
visualize_conv_layer(2,'re_lu',image_,5,6)
visualize_conv_layer(3,'lambda',image_,5,6)
visualize_conv_layer(4,'batch_normalization',image_,5,6)
visualize_conv_layer(5,'concatenate',image_,9,10)
visualize_conv_layer(6,'spatial_dropout2d',image_,9,10)
visualize_conv_layer(7,'conv2d_2',image_,5,6)
visualize_conv_layer(8,'re_lu_1',image_,5,6)
visualize_conv_layer(9,'lambda_1',image_,5,6)
visualize_conv_layer(10,'batch_normalization_1',image_,5,6)
visualize_conv_layer(11,'spatial_dropout2d_1',image_,5,6)
visualize_conv_layer(12,'conv2d_3',image_,5,6)
visualize_conv_layer(13,'re_lu_2',image_,5,6)
visualize_conv_layer(14,'lambda_2',image_,5,6)
visualize_conv_layer(15,'batch_normalization_2',image_,5,6)
visualize_conv_layer(16,'average_pooling2d',image_,5,6)
visualize_conv_layer(17,'spatial_dropout2d_2',image_,5,6)
visualize_conv_layer(18,'conv2d_4',image_,4,8)
visualize_conv_layer(19,'re_lu_3',image_,4,8)
visualize_conv_layer(20,'lambda_3',image_,4,8)
visualize_conv_layer(21,'batch_normalization_3',image_,4,8)
visualize_conv_layer(22,'concatenate_1',image_,8,12)
